In [1]:
import os
import requests
import json
import pandas as pd

In [2]:
url = "https://api.nasa.gov/planetary/apod" # the apod url

In [3]:
# parameters for the API
params = {
    'start_date': '',
    'end_date': '',
    'api_key': os.getenv('NASA_API_KEY')
}

In [4]:
# gets data from the url provided
response = requests.get(url, params=params)
response.status_code

200

In [5]:
# parse the JSON-formatted strin into a Python dictionary
data = json.loads(response.text)

In [6]:
# gotten from url
data

{'copyright': '\nPetr Horálek\n(ESO\nPhoto Ambassador,\nInst. of Physics in Opava) ;\n Acknowledgement: \nXavier Jubier\n',
 'date': '2024-03-31',
 'explanation': "In late 2021 there was a total solar eclipse visible only at the end of the Earth.  To capture the unusual phenomenon, airplanes took flight below the clouded seascape of Southern Ocean. The featured image shows one relatively spectacular capture where the bright spot is the outer corona of the Sun and the eclipsing Moon is seen as the dark spot in the center.  A wing and engine of the airplane are visible across the left and bottom of the image, while another airplane observing the eclipse is visible on the far left. The dark area of the sky surrounding the eclipsed Sun is called a shadow cone. It is dark because you are looking down a long corridor of air shadowed by the Moon. A careful inspection of the eclipsed Sun will reveal the planet Mercury just to the right. You won't have to travel to the end of the Earth to see t

## **NASA API USING THE START AND END DATE ON ANOTHER URL**

In [7]:
# working on another url
new_url = 'https://api.nasa.gov/neo/rest/v1/feed'

In [8]:
# parameters for new url
parameter = {
    'api_key': os.getenv('NASA_API_KEY'),
    'start_date': '2024-02-04',
    'end_date': '2024-02-09'
}

In [9]:
# used the requests library to get the data from the new url
new_response = requests.get(new_url, params=parameter)
new_response.status_code

200

In [ ]:
# parse the JSON-formatted strin into a Python dictionary
new_data = json.loads(new_response.text)
new_data

In [11]:
# the data gotten here appears to be in dictionary
new_data.keys()

dict_keys(['links', 'element_count', 'near_earth_objects'])

In [ ]:
new_data['near_earth_objects']

## **TASK**

Do the necessary data pre-processing tasks on the previous result in order to get a clean dataframe with the following columns:
- Asteroid ID
- Asteroid name
- The minimal estimated diameter in Kilometre
- Absolute magnitude
- Relative velocity(km/s)

In [13]:
# asteroid id
new_data['near_earth_objects']['2024-02-05'][0]['id']

'2347813'

In [14]:
# asteroid name
new_data['near_earth_objects']['2024-02-05'][0]['name']

'347813 (2002 NP1)'

In [15]:
# minimal estimated diameter in KM
new_data['near_earth_objects']['2024-02-05'][0]['estimated_diameter']['kilometers']['estimated_diameter_min']

0.7808272775

In [16]:
# absolute magnitude relative velocity(km/s)
new_data['near_earth_objects']['2024-02-05'][0]['absolute_magnitude_h']

17.66

In [17]:
# relative velocity(km/s)
new_data['near_earth_objects']['2024-02-05'][2]['close_approach_data'][0]['relative_velocity']['kilometers_per_second']

'8.2382303218'

## **PUT DATA INTO A DATAFRAME**

In [18]:
# create a DataFrame
df = pd.DataFrame() 
# loop through each day
for days in new_data['near_earth_objects']:
    # save the value for that day in a variable 'daily_near_earth_object'
    daily_near_earth_object = new_data['near_earth_objects'][days]
    # loop through the 'daily_near_earth_object'
    for near_earth_object in daily_near_earth_object:
        # create a temporary DataFrame 
        temp = pd.DataFrame(
            {
                'Asteroid_ID': [near_earth_object['id']],
                'Asteroid_name': [near_earth_object['name']],
                'Minimal_estimated_diameter(Km)': [near_earth_object['estimated_diameter']['kilometers']['estimated_diameter_min']],
                'Absolute_magnitude': [near_earth_object['absolute_magnitude_h']],
                'Relative_velocity(Km/s)': [near_earth_object['close_approach_data'][0]['relative_velocity']['kilometers_per_second']]
            }
        )
        # combine the temporary DataFrames
        df = pd.concat([df, temp], ignore_index=True)
df.head()

,Asteroid_ID,Asteroid_name,Minimal_estimated_diameter(Km),Absolute_magnitude,Relative_velocity(Km/s)
0,2347813,347813 (2002 NP1),0.780827,17.66,13.310416392
1,2444627,444627 (2006 WU),0.411675,19.05,1.9982604604
2,2451370,451370 (2011 AK5),0.134448,21.48,8.2382303218
3,3440771,(2008 WZ94),0.235806,20.26,16.1492348048
4,3557844,(2011 DW),0.076658,22.70,14.872591907


In [19]:
df.shape

(120, 5)